# Complete Crop-Seq Analysis Workflow

**Time:** 20-30 minutes  
**Level:** Intermediate  

## Learning Objectives

This tutorial covers a complete end-to-end Crop-Seq analysis workflow:

1. Data preprocessing and quality control
2. Guide extraction with parameter tuning
3. Comprehensive differential expression analysis
4. Advanced visualizations and interpretation
5. Results export and downstream analysis

## What You'll Learn

- How to preprocess Crop-Seq data properly
- Best practices for guide assignment
- Interpreting differential expression results
- Creating publication-ready figures
- Troubleshooting common issues

## Part 1: Setup and Data Loading

In [ ]:
# Import libraries
import perturbio as pt
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
sc.settings.set_figure_params(dpi=100, facecolor='white')
sns.set_style('whitegrid')

print(f"Perturbio version: {pt.__version__}")
print(f"Scanpy version: {sc.__version__}")

## Part 2: Load Your Data

In a real analysis, you would load your Crop-Seq data here. The data should be in AnnData format (h5ad file).

```python
# Load your actual data
adata = sc.read_h5ad("your_cropseq_data.h5ad")
```

For this tutorial, we'll create synthetic data:

In [ ]:
from scipy.sparse import csr_matrix

# Create realistic synthetic Crop-Seq data
np.random.seed(42)

n_cells = 500
n_genes = 200

# Guide RNAs
guides = [
    'BRCA1_guide1', 'BRCA1_guide2',
    'MYC_guide1', 'MYC_guide2',
    'TP53_guide1', 'TP53_guide2',
    'EGFR_guide1', 'KRAS_guide1',
    'non-targeting_1', 'non-targeting_2'
]

# Regular genes
regular_genes = [f'Gene_{i}' for i in range(n_genes - len(guides))]
all_genes = guides + regular_genes

# Create count matrix
X = csr_matrix(np.random.negative_binomial(5, 0.3, size=(n_cells, n_genes)).astype(np.float32))

# Create AnnData
adata = sc.AnnData(
    X=X,
    obs=pd.DataFrame(index=[f'Cell_{i}' for i in range(n_cells)]),
    var=pd.DataFrame(index=all_genes)
)

# Simulate guide expression patterns
cells_per_guide = 50
for i, guide in enumerate(guides):
    start_idx = i * cells_per_guide
    end_idx = start_idx + cells_per_guide
    if end_idx <= n_cells:
        guide_idx = list(adata.var_names).index(guide)
        adata.X[start_idx:end_idx, guide_idx] = np.random.poisson(15, size=cells_per_guide)

print(f"Loaded dataset: {adata.n_obs:,} cells × {adata.n_vars:,} genes")

## Part 3: Data Preprocessing and QC

Before running Perturbio, let's do some basic quality control with scanpy.

In [ ]:
# Calculate QC metrics
sc.pp.calculate_qc_metrics(adata, inplace=True)

# Plot QC metrics
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Total counts per cell
axes[0].hist(adata.obs['total_counts'], bins=50, color='steelblue', alpha=0.7)
axes[0].set_xlabel('Total counts per cell')
axes[0].set_ylabel('Number of cells')
axes[0].set_title('Library size distribution')

# Genes detected per cell
axes[1].hist(adata.obs['n_genes_by_counts'], bins=50, color='forestgreen', alpha=0.7)
axes[1].set_xlabel('Genes detected per cell')
axes[1].set_ylabel('Number of cells')
axes[1].set_title('Gene detection')

# Total counts vs genes detected
axes[2].scatter(adata.obs['total_counts'], adata.obs['n_genes_by_counts'], 
                s=5, alpha=0.5, color='coral')
axes[2].set_xlabel('Total counts')
axes[2].set_ylabel('Genes detected')
axes[2].set_title('Counts vs Genes')

plt.tight_layout()
plt.show()

print(f"\nQC Summary:")
print(f"  Median counts per cell: {adata.obs['total_counts'].median():.0f}")
print(f"  Median genes per cell: {adata.obs['n_genes_by_counts'].median():.0f}")

In [ ]:
# Optional: Filter low-quality cells
# sc.pp.filter_cells(adata, min_genes=50)
# sc.pp.filter_genes(adata, min_cells=3)

# Normalize and log-transform
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

print(f"✓ Data normalized and log-transformed")
print(f"  Data range: {adata.X.min():.2f} to {adata.X.max():.2f}")

## Part 4: Load Guide Library

Create or load your guide library describing which genes each guide targets.

In [ ]:
# Create guide library
guide_library = pd.DataFrame({
    'guide_id': [
        'BRCA1_guide1', 'BRCA1_guide2',
        'MYC_guide1', 'MYC_guide2',
        'TP53_guide1', 'TP53_guide2',
        'EGFR_guide1', 'KRAS_guide1',
        'non-targeting_1', 'non-targeting_2'
    ],
    'target_gene': [
        'BRCA1', 'BRCA1',
        'MYC', 'MYC',
        'TP53', 'TP53',
        'EGFR', 'KRAS',
        'control', 'control'
    ],
    'guide_sequence': [
        'GCACTCAGGAAACAGCTATG', 'CTGAAGACTGCTCAGTGTAG',
        'GTACTTGGTGAGGCCAGCGC', 'TACAGCGTGGTGGTGCCTAT',
        'CCATTGTTCAATATCGTCCG', 'GTCATAAAAGACGTTCTCCA',
        'GTTACGCCAAGCGGTTAGCG', 'GACGATACAGCTAATTCAGA',
        'GTAGCGAACGTGTCCGGCGT', 'ACGGAGGCTAAGCGTCGCAA'
    ]
})

# Save guide library
guide_library.to_csv('guide_library.csv', index=False)

print("Guide library:")
print(guide_library)
print(f"\n{len(guide_library)} guides total")
print(f"  Targeting: {(guide_library['target_gene'] != 'control').sum()}")
print(f"  Control: {(guide_library['target_gene'] == 'control').sum()}")

## Part 5: Step-by-Step Analysis

Now let's use Perturbio with more control over each step.

### 5.1: Extract Guide Barcodes

In [ ]:
# Initialize analyzer
analyzer = pt.CropSeqAnalyzer(adata)

# Extract guides with custom parameters
analyzer.extract_guides(
    guide_file='guide_library.csv',
    min_umis=3,  # Minimum UMIs to confidently assign a guide
    multiplet_threshold=0.2  # Threshold for detecting multiplets
)

In [ ]:
# Examine guide assignments
print("Guide assignment distribution:")
print(analyzer.adata.obs['perturbation'].value_counts())

# Check assignment confidence
print("\nAssignment confidence:")
print(analyzer.adata.obs['guide_confidence'].value_counts())

### 5.2: Differential Expression Analysis

In [ ]:
# Run differential expression
analyzer.differential_expression(
    control_label='control',
    min_cells=10,  # Minimum cells per perturbation
    fdr_threshold=0.05  # FDR threshold for significance
)

In [ ]:
# Access detailed results
de_results = analyzer.results.differential_expression

print(f"Total perturbations tested: {len(de_results.perturbations)}")
print(f"Perturbations: {de_results.perturbations}")

# Count significant genes per perturbation
sig_counts = de_results.genes_per_perturbation(fdr_threshold=0.05)
print("\nSignificant genes per perturbation:")
print(sig_counts)

## Part 6: Advanced Visualizations

### 6.1: Perturbation Assignment Overview

In [ ]:
# Plot cell counts per perturbation
fig = analyzer.plot_perturbation_counts()
plt.show()

### 6.2: Volcano Plots for Each Perturbation

In [ ]:
# Create volcano plots for top perturbations
perturbations_to_plot = de_results.perturbations[:4]  # Plot first 4

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for i, pert in enumerate(perturbations_to_plot):
    if i < len(axes):
        # Get DE results for this perturbation
        top_hits = analyzer.top_hits(pert, n=5)
        
        # Create volcano plot
        analyzer.plot_volcano(pert, fdr_threshold=0.05)
        
        print(f"\n{pert} - Top 5 hits:")
        print(top_hits[['gene', 'log_fc', 'pval_adj']].to_string(index=False))

plt.tight_layout()
plt.show()

### 6.3: Heatmap of Top Perturbed Genes

In [ ]:
# Create heatmap showing effect sizes across perturbations
# Get top 20 most significantly perturbed genes
sig_genes = de_results.significant_genes(fdr_threshold=0.05)
top_genes = sig_genes.nsmallest(20, 'pval_adj')['gene'].unique()

# Create matrix of log fold changes
heatmap_data = []
for pert in de_results.perturbations:
    pert_data = de_results.data[de_results.data['perturbation'] == pert]
    row = []
    for gene in top_genes:
        gene_data = pert_data[pert_data['gene'] == gene]
        if len(gene_data) > 0:
            row.append(gene_data['log_fc'].values[0])
        else:
            row.append(0)
    heatmap_data.append(row)

# Plot heatmap
fig, ax = plt.subplots(figsize=(12, 6))
sns.heatmap(
    heatmap_data,
    xticklabels=top_genes,
    yticklabels=de_results.perturbations,
    cmap='RdBu_r',
    center=0,
    cbar_kws={'label': 'Log2 Fold Change'},
    ax=ax
)
ax.set_title('Top Perturbed Genes Across Perturbations', fontsize=14, fontweight='bold')
ax.set_xlabel('Genes')
ax.set_ylabel('Perturbations')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## Part 7: Detailed Results Exploration

In [ ]:
# Compare effects of different guides targeting the same gene
# Example: BRCA1_guide1 vs BRCA1_guide2

if 'BRCA1' in de_results.perturbations:
    brca1_results = de_results.top_hits('BRCA1', n=20)
    
    print("Top 20 genes affected by BRCA1 perturbation:")
    print(brca1_results.to_string(index=False))
    
    # Check if target gene itself is knocked down
    target_check = brca1_results[brca1_results['gene'].str.contains('BRCA1', case=False, na=False)]
    if len(target_check) > 0:
        print("\n✓ Target gene BRCA1 is successfully knocked down!")
        print(target_check)

## Part 8: Export Results

Save everything for downstream analysis, sharing, or publication.

In [ ]:
# Export all results
output_dir = analyzer.export(output_dir='complete_workflow_results')

print(f"\n✓ Results exported to: {output_dir}")
print("\nContents:")
print("  📊 differential_expression.csv - All DE results")
print("  📊 perturbations.csv - Guide assignments per cell")
print("  📊 summary.txt - Human-readable summary")
print("  💾 annotated_data.h5ad - Data with perturbation info")
print("  📈 Figures - All plots in PNG format")

## Part 9: Downstream Analysis Tips

In [ ]:
# Access annotated data for custom scanpy analysis
adata_annotated = analyzer.adata

print("You can now use the annotated data with scanpy:")
print("\nExample analyses:")
print("  • Dimensionality reduction: sc.tl.pca(), sc.tl.umap()")
print("  • Clustering: sc.tl.leiden()")
print("  • Trajectory analysis: sc.tl.diffmap()")
print("  • Gene set enrichment: your favorite tool")
print("\nYour data now has:")
print(f"  • Perturbation labels: adata.obs['perturbation']")
print(f"  • Guide identities: adata.obs['guide_identity']")
print(f"  • DE results: adata.uns['perturbio_de']")

## Summary

In this tutorial, you learned:

✅ **Data preprocessing** - QC, normalization, and filtering  
✅ **Guide extraction** - With parameter tuning and validation  
✅ **Differential expression** - Comprehensive analysis workflow  
✅ **Visualizations** - Multiple plot types for interpretation  
✅ **Results export** - Saving for downstream analysis  
✅ **Quality control** - Checking guide assignments and DE results  

## Next Steps

- **Tutorial 03**: Advanced scanpy integration and custom workflows
- **Tutorial 04**: Command-line usage for batch processing

## Common Issues and Solutions

**Issue**: No guides detected  
**Solution**: Check that guide names in your library match gene names in your data

**Issue**: Low assignment rate  
**Solution**: Try lowering `min_umis` parameter or check guide library

**Issue**: No significant genes  
**Solution**: Check if you have enough cells per perturbation (min 10-20)

Happy analyzing! 🧬✨

In [ ]:
# Cleanup
import os
if os.path.exists('guide_library.csv'):
    os.remove('guide_library.csv')
print("Tutorial complete!")